In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

from avalanche.benchmarks.classic import CLStream51
#from avalanche.models import MobilenetV1
from pretrained_model.slda_resnet18 import SLDAResNetModel
from avalanche.training import StreamingLDA
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.logging.text_logging import TextLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    forgetting_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    ram_usage_metrics,
    timing_metrics,
)


c:\Users\USER\anaconda3\envs\avalanche-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Eval_metics
wandb_logger = WandBLogger(run_name="SLDA_stream51")
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    gpu_usage_metrics(gpu_id=0,experience=True),
    ram_usage_metrics(experience=True),
    timing_metrics(minibatch=True, epoch=True, experience=True),
    loggers=[InteractiveLogger(), wandb_logger,
                TextLogger(open('slda_results.txt', 'w'))]
)

# model
model = SLDAResNetModel(device=device)

# Prepare for training & testing
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

# CL Benchmark Creation
benchmark = CLStream51(scenario="class_instance", bbox_crop=True) 
# scenario
# "iid", "class_iid", "instance", "class_instance"

train_stream = benchmark.train_stream
test_stream = benchmark.test_stream

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jgpark (cais). Use `wandb login --relogin` to force relogin


E:\python\CL\avalanche_root\avalanche\training\plugins\evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


Shape mismatch...ignoring fc.weight
Shape mismatch...ignoring fc.bias
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Continual learning strategy
cl_strategy = StreamingLDA(
    model,
    criterion,
    input_size=512,
    num_classes=51,
    train_mb_size=128,
    eval_mb_size=128, device=device,
    evaluator=eval_plugin
)

# train and test loop over the stream of experiences
results = []
for train_exp in train_stream:
    cl_strategy.train(train_exp)
    results.append(cl_strategy.eval(test_stream))

-- >> Start of training phase << --
100%|██████████| 238/238 [06:40<00:00,  1.68s/it]
Epoch 0 ended.
	Time_Epoch/train_phase/train_stream/Task000 = 400.7542
	Time_MB/train_phase/train_stream/Task000 = 0.1693
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9780
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 4/4 [00:03<00:00,  1.17it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 796.8481
	MaxGPU0Usage_Experience/eval_phase/test_stream/Task000/Exp000 = 31.0000
	MaxRAMUsage_Experience/eval_phase/test_stream/Task000/Exp000 = 4387.8359
	Time_Exp/eval_phase/test_stream/Task000/Exp000 = 3.4177
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.8000
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 8/8 [00:06<00:00,  1.17it/s]
> Eval on exp